# Projet 4

In [32]:
import requests
import pandas as pd
import re
import json
import time

from bs4 import BeautifulSoup
from multiprocess import Pool, cpu_count
from bs4 import BeautifulSoup

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [41]:
url_1 = "https://kim.fspot.org/cours/page2.html"
url_2 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    result = {}
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    
    elements_price = soup.find_all(class_="pricing-table-header")
    elements_list = soup.find_all(class_="pricing-table-list")

    for i in range(len(elements_price)):
        category = elements_price[i].h2.text
        price = elements_price[i].find(string=re.compile("\$\d+")).strip()
        storage = elements_list[i].find(string=re.compile("storage"))
        storage = re.match("\d+\wB", storage)[0]
        database = elements_list[i].find(string=re.compile("database"))
        database = int(re.match("\d+", database)[0])
        result[category] = {'price': price, 'storage': storage,
                            'databases': database}
        
    return result

#### Résultats :

In [42]:
get_prices_from_url(url_1)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

In [43]:
get_prices_from_url(url_2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [28]:
url_3 = "https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33"

def extract_beer_infos(url):
    infos = {
        'name': None,
        'note': None,
        'price': None,
        'volume': None,
    }
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    
    name = soup.find("h1").text.strip()
    note = int(soup.find(class_="stars")["data-percent"])
    price = soup.find_all(string=re.compile("\d+,\d+\s€"))[0][:-2]
    price = float(price.replace(',', '.'))
    volume = soup.find(class_="js-beer-volume").text
    volume = int(re.match("\d+", volume)[0])
    
    infos['name'] = name
    infos['note'] = note
    infos['price'] = price
    infos['volume'] = volume
    
    return infos

#### Résultat : 

In [18]:
extract_beer_infos(url_3)

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [24]:
url_4 = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [33]:
def extract_beer_list_infos(url):
    
    blist = requests.get(url)
    blist = json.loads(blist.content)['items']
    
    # Collecter les pages de bières à partir du JSON
    beer_pages = ["https://www.beerwulf.com" + blist[i]['contentReference']
                  for i in range(len(blist))]
    
    # Sequential version (slow):
    time0 = time.time()
    beers = [extract_beer_infos(url) for url in beer_pages]
    time1 = time.time()
    print(f"Temps de traitement pour la version séquentielle : {time1-time0:.2f}s.")

    # Parallel version (faster):
    time0 = time.time()
    with Pool() as pool:
        beers = pool.map(extract_beer_infos, beer_pages)
    time1 = time.time()
    print(f"Temps de traitement pour la version parallélisée : {time1-time0:.2f}s.")
    
    return beers

#### Résultat :

In [34]:
extract_beer_list_infos(url_4)

Temps de traitement pour la version séquentielle : 2.52s.
Temps de traitement pour la version parallélisée : 0.76s.


[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay',
  'note': 60,
  'price': 38.99,
  'volume': 25},
 {'name': 'Brasserie De Sutter Brin de Folie',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'La Cristal IPA du Mont Blanc',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
 {'name': 'La Parisienne Le Titi Parisien',
  'note': 70,
  'price': 38.99,
  'volume': 33},
 {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
 {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
 {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
 {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
 {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]

In [44]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(url_1)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(url_2)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(url_4)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [45]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... 

Temps de traitement pour la version séquentielle : 2.09s.
Temps de traitement pour la version parallélisée : 0.83s.


ok

----------------------------------------------------------------------
Ran 3 tests in 3.303s

OK
